In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [2]:
kr_pm10_2015 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2015.csv", sep=',',encoding='mbcs')
kr_pm10_2016 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2016.csv", sep=',',encoding='mbcs')
kr_pm10_2017 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2017.csv", sep=',',encoding='mbcs')
kr_pm10_2018 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2018.csv", sep=',',encoding='mbcs')
kr_pm10_2019 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2019.csv", sep=',',encoding='mbcs')
kr_pm10_2020 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2020.csv", sep=',',encoding='mbcs')
kr_pm10_2021 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2021.csv", sep=',',encoding='mbcs')
kr_pm10_2022 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2022.csv", sep=',',encoding='mbcs')
kr_pm10_2023 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2023.csv", sep=',',encoding='mbcs')
kr_pm10_2024 = pd.read_csv(r".\Git\데이터\국내 pm10 평균농도\pm10_2024.csv", sep=',',encoding='mbcs')

In [3]:
kr_pm10 = pd.concat([kr_pm10_2015, kr_pm10_2016, kr_pm10_2017, kr_pm10_2018, kr_pm10_2019,
                    kr_pm10_2020, kr_pm10_2021, kr_pm10_2022, kr_pm10_2023, kr_pm10_2024], axis = 0)

In [4]:
kr_pm10 = kr_pm10.drop("지점", axis = 1)

In [5]:
kr_pm10.columns = ["지점명", "일시", "pm10"]

In [6]:
kr_pm10["지점명"].unique()

array(['속초', '광덕산', '대관령', '춘천', '백령도', '서울', '울릉도독도(감)', '관악산', '수원',
       '영월', '울진', '안면도(감)', '추풍령', '안동', '군산', '대구', '전주', '울산', '광주',
       '구덕산', '흑산도', '진도(첨찰산)', '고산', '진주', '강화', '북격렬비도', '천안', '진도군',
       '문경', '연평도', '북춘천'], dtype=object)

## 종속변수 지점 선택

- 추후 어느 지점들을 종속변수로 삼을지 논의가 필요함
    - 가능하다면 초미세먼지 데이터가 있는 곳과 중복되도록 하는게 좋을지도
    - 현재 초미세먼지는 서울만 예측을 하고 여유가 된다면 다른 지역도 시도해보는 방향

- 종속변수 지정을 위해 고려할만한 요소들?
    - 인구가 많은 지점
    - 특정 지역을 대표할 수 있는 지점
    - 관광객 등의 유동인구나 관심이 높은 지점
    
- 종속변수마다 모델을 새로 만들고 학습시켜야 하는 굉장히 복잡한 과정을 거쳐야함
    - 현실적으로 너무 많은 지점을 종속변수로 채택할 수 없음
    - 경기도, 강원도, 충청도, 전라도, 경상도를 대표하는 지점 1개씩 지정하는 방향 고민중

- 각 지점별 메모
    - 수도권 대표로 **서울**
    - **구덕산**은 부산에 있는 산. 경상도 대표로 선정하기 위해서는 구덕산이 적절한 것인지, **울산**으로 대체할 것인지 고민 필요
        - 구덕산의 기상레이더관측소는 고도 391m의 시약산 정상에 위치해있음. 부산의 미세먼지를 반영하기에 적절한가?
        - https://www.newsis.com/view/NISX20180114_0000202464
    - 충청도 대표 **천안**...? 세종이나 대전 있으면 좋을텐데...
    - 전라도 대표 **광주**
    - 강원도 도시 지점인 속초, 춘천, 영월 중에서 인구가 가장 많은 지점은 **춘천**
        - 동해안의 대표로써 **속초**가 지리적으로 적절하다고 생각됨

In [7]:
# kr_pm10의 일시 피처를 datetime 형식으로 변환
kr_pm10["일시"] = kr_pm10["일시"].map(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M'))

In [8]:
# pivot 적용
kr_pm10_pivot = kr_pm10.pivot(index = ["일시"], columns = ["지점명"], values = ["pm10"])

In [9]:
kr_pm10_pivot.shape

(82143, 31)

In [10]:
# 5개 지점 데이터만 추출
kr_pm10_pivot = kr_pm10_pivot.loc[:, [("pm10", "서울"), ("pm10", "속초"), ("pm10", "천안"), ("pm10", "광주"), ("pm10", "울산")]]

In [14]:
kr_pm10_pivot.head()

pm10                        
지점명                     서울    속초    천안    광주    울산
일시                                                
2015-01-01 00:00:00   57.0  25.0  47.0  11.0  11.0
2015-01-01 01:00:00   51.0  22.0  38.0  13.0  10.0
2015-01-01 02:00:00   72.0  33.0  45.0  16.0  13.0
2015-01-01 03:00:00   88.0  27.0  64.0  17.0  17.0
2015-01-01 04:00:00  106.0  30.0  75.0  21.0  20.0

### 결측된 시간 데이터 추가

In [15]:
# 결측된 시간을 kr_pm10_pivot에 추가하기 위해 동일한 형식을 갖는 데이터프레임 생성
time_df_1h = pd.DataFrame([kr_pm10_pivot.index[0] + timedelta(hours = 1) * i for i in range(((366 * 2) + (365 * 7) + (31 + 29 + 31 + 30 + 19)) * 24)])
time_df_1h.columns = ["일시"]
time_df_1h = time_df_1h.set_index("일시")

In [17]:
# 동일한 컬럼 명칭을 갖는 5개의 컬럼 생성. 값은 전부 NaN
time_df_1h["0"] = np.nan
time_df_1h["1"] = np.nan
time_df_1h["2"] = np.nan
time_df_1h["3"] = np.nan
time_df_1h["4"] = np.nan
time_df_1h.columns = kr_pm10_pivot.columns

In [28]:
# kr_pm10_pivot에 없는 시간만 남기고 제거
time_df_1h = time_df_1h[~time_df_1h.index.isin(kr_pm10_pivot.index)]

In [29]:
# kr_pm10_pivot과 concat
kr_pm10_pivot = pd.concat([kr_pm10_pivot, time_df_1h], axis = 0)
kr_pm10_pivot.tail()

pm10                
지점명                   서울  속초  천안  광주  울산
일시                                      
2022-10-18 04:00:00  NaN NaN NaN NaN NaN
2022-10-18 05:00:00  NaN NaN NaN NaN NaN
2022-10-18 06:00:00  NaN NaN NaN NaN NaN
2022-11-03 15:00:00  NaN NaN NaN NaN NaN
2023-11-09 00:00:00  NaN NaN NaN NaN NaN

In [32]:
# 일시 순으로 맞도록 정렬
kr_pm10_pivot = kr_pm10_pivot.sort_index()

In [34]:
kr_pm10_pivot.tail()

pm10                        
지점명                    서울    속초    천안    광주    울산
일시                                               
2024-05-19 19:00:00  24.0  34.0  38.0  49.0  26.0
2024-05-19 20:00:00  21.0  23.0  50.0  30.0  34.0
2024-05-19 21:00:00  17.0  26.0  17.0  17.0  32.0
2024-05-19 22:00:00  18.0  23.0  23.0   8.0  29.0
2024-05-19 23:00:00  20.0  26.0  21.0   NaN  29.0

In [38]:
kr_pm10_pivot.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 82248 entries, 2015-01-01 00:00:00 to 2024-05-19 23:00:00
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   (pm10, 서울)  77079 non-null  float64
 1   (pm10, 속초)  77128 non-null  float64
 2   (pm10, 천안)  77396 non-null  float64
 3   (pm10, 광주)  75839 non-null  float64
 4   (pm10, 울산)  78617 non-null  float64
dtypes: float64(5)
memory usage: 3.8 MB


- 4000 ~ 7000개의 결측치가 있음
- 3시간씩 평균으로 묶은 이후에도 결측치가 있다면 해당 데이터는 폐기할 예정

### 3시간 단위로 데이터 변환

In [35]:
# 3시간 단위로 묶기 위한 신규 데이터프레임 생성, 컬럼명 변경
time_df_3h = pd.DataFrame([kr_pm10_pivot.index[0] + timedelta(hours = 3) * i for i in range(((366 * 2) + (365 * 7) + (31 + 29 + 31 + 30 + 19)) * 8)])
time_df_3h.columns = ["일시"]

In [41]:
time_df_3h = time_df_3h.set_index("일시")

In [42]:
# 동일한 컬럼 명칭을 갖는 5개의 컬럼 생성. 값은 전부 NaN
time_df_3h["0"] = np.nan
time_df_3h["1"] = np.nan
time_df_3h["2"] = np.nan
time_df_3h["3"] = np.nan
time_df_3h["4"] = np.nan
time_df_3h.columns = kr_pm10_pivot.columns

In [79]:
# 15년 1월 1일 00시를 제외한 나머지 나머지 시간대에 kr_pm10_pivot의 데이터를 입력하기
for i in range(1, len(time_df_3h)):
    for loc in time_df_3h.columns:
        # time_df_3h의 시간 time2와 각각 2시간, 1시간 전의 시간 time0, time1 지정
        time0 = time_df_3h.index[i] - timedelta(hours = 2)
        time1 = time_df_3h.index[i] - timedelta(hours = 1)
        time2 = time_df_3h.index[i]
        
        # 각각의 시간, 지점의 pm10 농도 값을 지정
        value0 = kr_pm10_pivot.loc[time0, loc]
        value1 = kr_pm10_pivot.loc[time1, loc]
        value2 = kr_pm10_pivot.loc[time2, loc]
        
        # 3개의 value를 하나의 리스트에 담고 해당 리스트에서 NaN값을 제거한 value_list2 생성
        value_list = [value0, value1, value2]
        value_list2 = [i for i in value_list if pd.isna(i) == False]
        
        # value_list2가 빈 리스트라면 평균값은 NaN
        if len(value_list2) == 0:
            mean = np.nan
        
        # 1개 이상의 값이 있다면 해당 리스트의 평균, 반올림 0까지 처리하여 평균값 지정
        else:
            mean = round(np.mean(value_list2), 0)
        
        # time_df_3h의 시간과 지점이 일치하는 위치에 평균값 입력
        time_df_3h.loc[time2, loc] = mean

In [83]:
# 15년 1월 1일 00시 데이터 입력
time_df_3h.iloc[0, :] = kr_pm10_pivot.iloc[0, :]

In [86]:
time_df_3h.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 27416 entries, 2015-01-01 00:00:00 to 2024-05-19 21:00:00
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   (pm10, 서울)  26467 non-null  float64
 1   (pm10, 속초)  26584 non-null  float64
 2   (pm10, 천안)  26603 non-null  float64
 3   (pm10, 광주)  26309 non-null  float64
 4   (pm10, 울산)  26970 non-null  float64
dtypes: float64(5)
memory usage: 2.3 MB


- 여전히 지점별로 최대 1000개의 결측치가 있는 것이 파악됨

In [98]:
time_df_3h.to_csv("./kr_pm10.csv")